In [10]:
import json
import re


#flair_text_path = r"C:\Users\hp\Downloads\clever contact\clevercontracts\Contracts\2MAINS\test_predictions.txt"
#roberta_json_path = r"C:\Users\hp\Downloads\clever contact\clevercontracts\Contracts\2MAINS\roberta_predictions.json"
#flair_json_path = r"C:\Users\hp\Downloads\clever contact\flair_finetuned_entities_fulltext3.json"
#roberta_json_path = r"C:\Users\hp\Downloads\clever contact\roberta_predictions_finetuned3.json"
flair_json_path = r"C:\Users\hp\Downloads\clever contact\flair_finetuned_entities_fulltext4.json"
roberta_json_path = r"C:\Users\hp\Downloads\clever contact\roberta_predictions_finetuned4.json"

# Read Flair output text file
#with open(flair_text_path, "r", encoding="utf-8") as f:
    #flair_text = f.read()
with open(flair_json_path, "r", encoding="utf-8") as f:
    flair_text = json.load(f)
# Extract entities using regex pattern `<entity>/TYPE`
#flair_entities = []
#entity_pattern = re.compile(r'"(.+?)"/(LOC|ORG|PER|MISC)')

flair_entities = []
for item in flair_text:
    flair_entities.append({
        "text": item.get("text"),
        "type": item.get("type"),
        "start": item.get("start"),
        "end": item.get("end")
    })


# Iterate over matches and store them as structured dictionaries
#for match in entity_pattern.finditer(flair_text):
   # flair_entities.append({
      #  "text": match.group(1),  # Entity text
      #  "type": match.group(2),  # Entity type (LOC, ORG, PER, MISC)
       # "start": None,  # Start position not available from text format
       # "end": None     # End position not available from text format
   # })

print(f" Extracted {len(flair_entities)} entities from Flair output!")

# Load RoBERTa results from JSON
with open(roberta_json_path, "r", encoding="utf-8") as f:
    roberta_entities = json.load(f)

print(f" Loaded {len(roberta_entities)} entities from RoBERTa output!")


 Extracted 295 entities from Flair output!
 Loaded 902 entities from RoBERTa output!


In [11]:
def merge_ner_results(flair_results, roberta_results):
    """ Merges Flair and RoBERTa results by matching entity text. """
    merged_entities = []
    entity_dict = {}

    for entity in flair_results + roberta_results:
        key = entity["text"]  # Match by entity text only

        if key in entity_dict:
            existing = entity_dict[key]
            if entity["type"] == existing["type"]:  # Same type → Keep the first one
                entity_dict[key] = entity
        else:
            entity_dict[key] = entity  # Add new entity

    return list(entity_dict.values())

# Merge Flair & RoBERTa
final_entities = merge_ner_results(flair_entities, roberta_entities)

# Save merged results
merged_output_path = r"C:\Users\hp\Downloads\clever contact\merged_ner_predictions4.json"

with open(merged_output_path, "w", encoding="utf-8") as f:
    json.dump(final_entities, f, ensure_ascii=False, indent=4)

print(f" Merged NER results saved in {merged_output_path}!")


print(" Final Merged Entities:")
for entity in final_entities:
    print(f"Entity: {entity['text']} | Type: {entity['type']}")


 Merged NER results saved in C:\Users\hp\Downloads\clever contact\merged_ner_predictions4.json!
 Final Merged Entities:
Entity: SECRETARIAT GENERAL DIRECTION DES FINANCES | Type: ORG
Entity: VALORISATION DES DONNEES CAHIER DES CLAUSES ADMINISTRATIVES PARTICULIÈRES | Type: LOC
Entity: Code de la commande publique ACHETEUR MINISTERES SOCIAUX SECRETARIAT GENERAL DIRECTION DU NUMERIQUE  CCAP BPM060225_DATAVAL BPM060225 2 | Type: MISC
Entity: Acheteur public | Type: MISC
Entity: Direction du numérique | Type: ORG
Entity: Département ministériel Partie | Type: ORG
Entity: Directions des Ministères sociaux | Type: ORG
Entity: MiTN Mission transformation numérique ARS Agences régionales de santé DGS Direction Générale de la Santé DGOS Direction Générale de l'Offre de Soins | Type: MISC
Entity: Délais Sexpriment | Type: MISC
Entity: Clauses Administratives Particulières RH Ressources Humaines Mission Prestation | Type: MISC
Entity: Professionnelle | Type: MISC
Entity: Ministères sociaux Directi

In [12]:
def get_common_entities(flair_results, roberta_results):
    """Extracts only entities that are common between Flair and RoBERTa."""
    common_entities = []
    
    # Convert RoBERTa entities into a dictionary 
    roberta_dict = { (entity["text"].lower(), entity["type"]) for entity in roberta_results }

    # Compare each Flair entity to RoBERTa
    for entity in flair_results:
        key = (entity["text"].lower(), entity["type"])  # Normalize case
        if key in roberta_dict:  # Entity exists in both Flair and RoBERTa
            common_entities.append(entity)

    return common_entities


In [13]:
# Find common entities
common_entities = get_common_entities(flair_entities, roberta_entities)


print(" Common Entities Found in Both Flair & RoBERTa:")
for entity in common_entities:
    print(f"Entity: {entity['text']} | Type: {entity['type']}")

# Save common results to a new JSON file
common_output_path = r"C:\Users\hp\Downloads\clever contact\common_ner_predictions4.json"

with open(common_output_path, "w", encoding="utf-8") as f:
    json.dump(common_entities, f, ensure_ascii=False, indent=4)

print(f" Common NER results saved in {common_output_path}!")


 Common Entities Found in Both Flair & RoBERTa:
Entity: Les | Type: LOC
Entity: Les | Type: LOC
Entity: Les | Type: LOC
 Common NER results saved in C:\Users\hp\Downloads\clever contact\common_ner_predictions4.json!
